In [ ]:

!pip install shap seaborn
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


url = 'https://raw.githubusercontent.com/CharlesCLuo/Application-of-AI-in-Supply-Chain-Risk-Management-Series/main/Demand_Forecsting/MLP_weekly_demand.csv'
data = pd.read_csv(url)


original_data = data.copy()


print("Data preview:")
print(data.head())


descriptive_stats = data.describe()


columns_to_round = ['historical_sales', 'temperature', 'weekly_demand']
descriptive_stats[columns_to_round] = descriptive_stats[columns_to_round].round(2)


print("Descriptive Statistics (Rounded):")
print(descriptive_stats)


data.hist(figsize=(10, 10))
plt.suptitle('Figure 4.3: Feature Distributions')
plt.show()


plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Figure 4.4: Correlation Heatmap')
plt.show()


X = data[['historical_sales', 'promotion', 'temperature', 'precipitation', 'regional_gdp']].values
y = data['weekly_demand'].values


X_original = X.copy()


scaler = StandardScaler()
X = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = tf.keras.Sequential([
layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
layers.Dense(16, activation='relu'),
layers.Dense(1)
])


model.compile(optimizer='adam', loss='mse', metrics=['mae'])


history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=16)


plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.title('Figure 4.5: Training and Validation Loss Over Epochs')
plt.show()


y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R^2 Score: {r2}')


plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel('Actual Weekly Demand')
plt.ylabel('Predicted Weekly Demand')
plt.title('Figure 4.6: Actual vs Predicted Weekly Demand')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.show()


residuals = y_test - y_pred.flatten()
plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Weekly Demand')
plt.ylabel('Residuals')
plt.title('Figure 4.7: Residuals Plot')
plt.show()


plt.figure(figsize=(10, 6))
plt.hist(y_test, bins=30, alpha=0.5, label='Actual', color='blue')
plt.hist(y_pred, bins=30, alpha=0.5, label='Predicted', color='orange')
plt.xlabel('Weekly Demand')
plt.ylabel('Frequency')
plt.title('Figure 4.8: Distribution of Actual vs Predicted Weekly Demand')
plt.legend()
plt.show()


explainer = shap.KernelExplainer(model.predict, X_train)
shap_values = explainer.shap_values(X_test, nsamples=100)


X_test_df = pd.DataFrame(X_test, columns=['historical_sales', 'promotion', 'temperature', 'precipitation', 'regional_gdp'])


shap.summary_plot(shap_values, features=X_test_df, show=False)
plt.title('Figure 4.9: SHAP Summary Plot')
plt.show()
